# Poisson Regression Testing (v1.0)

Casey A Graff

September 7th, 2017

Developing and initial exploration of results from poisson regression.

In [ ]:
REP_DIR = "/home/cagraff/Documents/dev/fire_prediction/"
SRC_DIR = REP_DIR + 'src/'
DATA_DIR = REP_DIR + 'data/'

# Load system-wide packages
import os
import sys
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from collections import defaultdict
import datetime as dt
plt.rcParams['figure.figsize'] = [15,7]
%matplotlib inline

# Load project packages
os.chdir(SRC_DIR)
from features.loaders import load_integrated_df
import models.poisson_regression as pr
import models.linear_regression as lr
import models.evaluation as ev
from models import metrics
from helper import date_util as du

In [ ]:
sys.path.append(SRC_DIR+'features')
int_5km_10days_14_df = load_integrated_df(os.path.join(DATA_DIR, 'interim/integrated/fire_weather/fire_weather_integrated_gfs_modis_5km_10days_1400_alaska_2007-2016.pkl'))
int_5km_10days_14_4rain_df = load_integrated_df(os.path.join(DATA_DIR, 'interim/integrated/fire_weather/fire_weather_integrated_gfs_modis_5km_10days_1400_rain-del-4_alaska_2007-2016.pkl'))
int_5km_10days_14_8rain_df = load_integrated_df(os.path.join(DATA_DIR, 'interim/integrated/fire_weather/fire_weather_integrated_gfs_modis_5km_10days_1400_rain-del-8_alaska_2007-2016.pkl'))
int_5km_10days_8_df = load_integrated_df(os.path.join(DATA_DIR, 'interim/integrated/fire_weather/fire_weather_integrated_gfs_modis_5km_10days_0800_alaska_2007-2016.pkl'))
int_5km_10days_14_4raina_df = load_integrated_df(os.path.join(DATA_DIR, 'interim/integrated/fire_weather/fire_weather_integrated_gfs_modis_5km_10days_1400_rain-ahead-4_alaska_2007-2016.pkl'))

In [ ]:
int_5km_10days_14_df

In [ ]:
reload(pr)
reload(lr)
reload(ev)

def train(X, t_k_arr, leave_one_out=True, years=None):
    # Standardize weather
    for cov in ['temperature', 'humidity', 'wind', 'rain']:
        X[cov] = (X[cov] - np.mean(X[cov])) / np.var(X[cov])

    X = X.assign(year=map(lambda x: x.year, X.date_local))
    
    # Filter years
    if years:
        X = X[X.year.isin(years)]
    
    results_dict = defaultdict(list)
    for t_k in t_k_arr:
        print 'Starting t_k=%d' % t_k

        # Filter out predicting before fire started
        legit_series = pd.Series(index=X.index)
        for clust in X.cluster_id.unique():
            clust_df = X[X.cluster_id==clust]
            legit_day = np.min(clust_df.date_local) + du.INC_ONE_DAY * (t_k+1)
            legit_series[clust_df[clust_df.date_local>=legit_day].index]=1        

        X_legit = X[legit_series==1]

        X_t = pr.PoissonRegressionModel(t_k, []).add_autoregressive_col(X_legit, t_k+1)

        results_dict['baseline'].append((X_t.num_det, X_t.num_det_prev))

        prm = pr.PoissonRegressionModel(t_k=t_k, covariates=[])
        if leave_one_out:
            results, years = ev.cross_validation_years(prm, X_t)
        else:
            results, years = ev.leave_none_out(prm, X_t)
        results_dict['auto'].append(np.concatenate(results, axis=1))

        prm = pr.PoissonRegressionModel(t_k=t_k, covariates=['temperature', 'humidity'])
        if leave_one_out:
            results, years = ev.cross_validation_years(prm, X_t)
        else:
            results, years = ev.leave_none_out(prm, X_t)
        results_dict['temp_humid'].append(np.concatenate(results, axis=1))

        prm = pr.PoissonRegressionModel(t_k=t_k, covariates=['temperature', 'humidity', 'wind', 'rain'])
        if leave_one_out:
            results, years = ev.cross_validation_years(prm, X_t)
        else:
            results, years = ev.leave_none_out(prm, X_t)
        results_dict['all'].append(np.concatenate(results, axis=1)) 
        
        prm = pr.PoissonRegressionModel(t_k=t_k, covariates=['temperature', 'wind', 'rain'])
        if leave_one_out:
            results, years = ev.cross_validation_years(prm, X_t)
        else:
            results, years = ev.leave_none_out(prm, X_t)
        results_dict['no_humid'].append(np.concatenate(results, axis=1)) 
        
        prm = pr.PoissonRegressionModel(t_k=t_k, covariates=['temperature'])
        if leave_one_out:
            results, years = ev.cross_validation_years(prm, X_t)
        else:
            results, years = ev.leave_none_out(prm, X_t)
        results_dict['temp'].append(np.concatenate(results, axis=1)) 
     
        """
        lrm = lr.LinearRegressionModel(t_k=t_k, covariates=[])
        if leave_one_out:
            results, years = ev.cross_validation_years(lrm, X_t)
        else:
            results, years = ev.leave_none_out(prm, X_t)
        results_dict['auto_linear'].append(np.concatenate(results, axis=1))
        
        lrm = lr.LinearRegressionModel(t_k=t_k, covariates=['temperature', 'humidity', 'wind', 'rain'])
        if leave_one_out:
            results, years = ev.cross_validation_years(lrm, X_t)
        else:
            results, years = ev.leave_none_out(prm, X_t)
        results_dict['linear_all'].append(np.concatenate(results, axis=1))
        """
    return results_dict
        
def plot_training(results, t_k_arr, metric=metrics.mean_absolute_error):
    plt.plot(t_k_arr+1, map(lambda x: metric(*x), results['baseline']), "kv--", label="Baseline", linewidth=2)
    plt.plot(t_k_arr+1, map(lambda x: metric(*x), results['auto']), "gs--", label="Autoregression", linewidth=2)
    plt.plot(t_k_arr+1, map(lambda x: metric(*x), results['temp_humid']), "r^--", label="Temp/hum", linewidth=2)
    plt.plot(t_k_arr+1, map(lambda x: metric(*x), results['all']), "bo--", label="All weather", linewidth=2)
    plt.plot(t_k_arr+1, map(lambda x: metric(*x), results['no_humid']), "co--", label="All (except humid)", linewidth=2)
    plt.plot(t_k_arr+1, map(lambda x: metric(*x), results['temp']), "yo--", label="Temp Only", linewidth=2)
    #plt.plot(t_k_arr+1, map(lambda x: metric(*x), results['auto_linear']), "cs--", label='Linear Autoregression', linewidth=2)
    #plt.plot(t_k_arr+1, map(lambda x: metric(*x), results['linear_all']), "yo--", label='Linear All', linewidth=2)

    plt.rcParams.update({'font.size': 14})
    lgd = plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.xlabel("Day of forecast (k)")
    plt.xticks(t_k_arr+1)
    plt.ylabel(metric.__name__)
    
def print_summary(X, cov_list, t_k):
    # Standardize weather
    for cov in cov_list:
        X[cov] = (X[cov] - np.mean(X[cov])) / np.var(X[cov])

    X = X.assign(year=map(lambda x: x.year, X.date_local))
    X_t = pr.PoissonRegressionModel(t_k, []).add_autoregressive_col(X, t_k+1)

    prm = pr.PoissonRegressionModel(t_k=t_k, covariates=cov_list)
    results, years = ev.cross_validation_years(prm, X_t)
    print 'Mean Abs. Error: %f' % metrics.mean_absolute_error(*np.concatenate(results, axis=1))
    print prm.fit_result.summary()

## Comparing Clusterings on Ten Year (out of sample)

In [ ]:
t_k_arr = np.arange(0, 5)

results = train(int_5km_10days_14_df.copy(), t_k_arr)
plot_training(results, t_k_arr, metrics.mean_absolute_error)
plt.show()
plot_training(results, t_k_arr, metrics.root_mean_squared_error)

In [ ]:
t_k_arr = np.arange(0, 5)

results = train(int_5km_10days_14_4rain_df.copy(), t_k_arr)
plot_training(results, t_k_arr, metrics.mean_absolute_error)
plt.show()
plot_training(results, t_k_arr, metrics.root_mean_squared_error)

In [ ]:
t_k_arr = np.arange(0, 5)

results = train(int_5km_10days_14_8rain_df.copy(), t_k_arr)
plot_training(results, t_k_arr, metrics.mean_absolute_error)
plt.show()
plot_training(results, t_k_arr, metrics.root_mean_squared_error)

In [ ]:
t_k_arr = np.arange(0, 5)

results = train(int_5km_10days_14_4raina_df.copy(), t_k_arr)
plot_training(results, t_k_arr, metrics.mean_absolute_error)
plt.show()
plot_training(results, t_k_arr, metrics.root_mean_squared_error)

## Comparing Clusterings on Ten Year (in sample)

In [ ]:
t_k_arr = np.arange(0, 5)

results = train(int_5km_10days_14_df.copy(), t_k_arr, False)
plot_training(results, t_k_arr, metrics.mean_absolute_error)
plt.show()
plot_training(results, t_k_arr, metrics.root_mean_squared_error)

In [ ]:
t_k_arr = np.arange(0, 5)

results = train(int_5km_10days_14_4rain_df.copy(), t_k_arr, False)
plot_training(results, t_k_arr, metrics.mean_absolute_error)
plt.show()
plot_training(results, t_k_arr, metrics.root_mean_squared_error)

In [ ]:
t_k_arr = np.arange(0, 5)

results = train(int_5km_10days_14_8rain_df.copy(), t_k_arr, False)
plot_training(results, t_k_arr, metrics.mean_absolute_error)
plt.show()
plot_training(results, t_k_arr, metrics.root_mean_squared_error)

## Try a Different Time of Day (0800 instead of 1400)

In [ ]:
t_k_arr = np.arange(0, 5)

results = train(int_5km_10days_8_df.copy(), t_k_arr)

plot_training(results, t_k_arr, metrics.mean_absolute_error)
plt.show()
plot_training(results, t_k_arr, metrics.root_mean_squared_error)

In [ ]:
t_k_arr = np.arange(0, 5)

results = train(int_5km_10days_8_df.copy(), t_k_arr, False)

plot_training(results, t_k_arr, metrics.mean_absolute_error)
plt.show()
plot_training(results, t_k_arr, metrics.root_mean_squared_error)

In [ ]:
reload(pr)
reload(ev)

X = int_1200_df.copy()
t_k_arr = np.arange(0, 5)

# Standardize weather
for cov in ['temperature', 'humidity', 'wind', 'rain']:
    X[cov] = (X[cov] - np.mean(X[cov])) / np.var(X[cov])
    
X = X.assign(year=map(lambda x: x.year, X.date_local))

X = X[(X.year==2010)]


results_baseline, results_auto, results_temp_humid, results_all = [], [], [], []
for t_k in t_k_arr:
    print 'Starting t_k=%d' % t_k
    """
    # Filter out predicting before fire started
    legit_series = pd.Series(index=X.index)
    for clust in X.cluster_id.unique():
        clust_df = X[X.cluster_id==clust]
        legit_day = np.min(clust_df.date_local) + du.INC_ONE_DAY * (t_k+1)
        legit_series[clust_df[clust_df.date_local>=legit_day].index]=1
    
    X = X[legit_series==1]
    """
    X_t = pr.PoissonRegressionModel(t_k, []).add_autoregressive_col(X, t_k+1)
    
    results_baseline.append(metrics.mean_absolute_error(X_t.num_det, np.exp(X_t.num_det_prev_log)-1))
    
    prm = pr.PoissonRegressionModel(t_k=t_k, covariates=[])
    results, years = ev.leave_none_out(prm, X_t)
    results_auto.append(metrics.mean_absolute_error(*np.concatenate(results, axis=1)))
    
    prm = pr.PoissonRegressionModel(t_k=t_k, covariates=['temperature', 'humidity'])
    results, years = ev.leave_none_out(prm, X_t)
    results_temp_humid.append(metrics.mean_absolute_error(*np.concatenate(results, axis=1)))
    
    prm = pr.PoissonRegressionModel(t_k=t_k, covariates=['temperature', 'humidity', 'wind', 'rain'])
    results, years = ev.leave_none_out(prm, X_t)
    results_all.append(metrics.mean_absolute_error(*np.concatenate(results, axis=1)))   

In [ ]:
plt.plot(t_k_arr+1, results_baseline, "kv--", label="Baseline", linewidth=2)
plt.plot(t_k_arr+1, results_auto, "gs--", label="Autoregression", linewidth=2)
plt.plot(t_k_arr+1, results_temp_humid, "r^--", label="Temp/hum", linewidth=2)
plt.plot(t_k_arr+1, results_all, "bo--", label="All weather", linewidth=2)

plt.rcParams.update({'font.size': 14})
lgd = plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.xlabel("Day of forecast (k)")
plt.xticks(t_k_arr+1)
plt.ylabel("Mean absolute error (2007-2016)")

## Compare with old data

In [ ]:
clust_df = load_integrated_df(os.path.join(DATA_DIR, 'archived/cluster/clust_feat_df_5.pkl'))

In [ ]:
# Modify old data to work with new code
clust_df['date_local'] = np.array(map(lambda x: dt.datetime(int(x), 1, 1), clust_df.year)) + np.array(map(lambda x: dt.timedelta(int(x) - 1), clust_df.dayofyear))
clust_df.rename(columns={'temp': 'temperature', 'cluster': 'cluster_id', 'n_det': 'num_det'}, inplace=True)

In [ ]:
reload(pr)
reload(ev)

X = clust_df.copy()
t_k_arr = np.arange(0, 5)

# Standardize weather
for cov in ['temperature', 'humidity', 'wind', 'rain']:
    X[cov] = (X[cov] - np.mean(X[cov])) / np.var(X[cov])
    
X = X.assign(year=map(lambda x: x.year, X.date_local))



results_baseline, results_auto, results_temp_humid, results_all = [], [], [], []
for t_k in t_k_arr:
    print 'Starting t_k=%d' % t_k
    """
    # Filter out predicting before fire started
    legit_series = pd.Series(index=X.index)
    for clust in X.cluster_id.unique():
        clust_df = X[X.cluster_id==clust]
        legit_day = np.min(clust_df.date_local) + du.INC_ONE_DAY * (t_k+1)
        legit_series[clust_df[clust_df.date_local>=legit_day].index]=1
    
    X = X[legit_series==1]
    """
    X_t = pr.PoissonRegressionModel(t_k, []).add_autoregressive_col(X, t_k+1)
    
    results_baseline.append(metrics.mean_absolute_error(X_t.num_det, np.exp(X_t.num_det_prev_log)-1))
    
    prm = pr.PoissonRegressionModel(t_k=t_k, covariates=[])
    results, years = ev.cross_validation_years(prm, X_t)
    results_auto.append(metrics.mean_absolute_error(*np.concatenate(results, axis=1)))
    
    prm = pr.PoissonRegressionModel(t_k=t_k, covariates=['temperature', 'humidity'])
    results, years = ev.cross_validation_years(prm, X_t)
    results_temp_humid.append(metrics.mean_absolute_error(*np.concatenate(results, axis=1)))
    
    prm = pr.PoissonRegressionModel(t_k=t_k, covariates=['temperature', 'humidity', 'wind', 'rain'])
    results, years = ev.cross_validation_years(prm, X_t)
    results_all.append(metrics.mean_absolute_error(*np.concatenate(results, axis=1)))   

In [ ]:
plt.plot(t_k_arr+1, results_baseline, "kv--", label="Baseline", linewidth=2)
plt.plot(t_k_arr+1, results_auto, "gs--", label="Autoregression", linewidth=2)
plt.plot(t_k_arr+1, results_temp_humid, "r^--", label="Temp/hum", linewidth=2)
plt.plot(t_k_arr+1, results_all, "bo--", label="All weather", linewidth=2)

plt.rcParams.update({'font.size': 14})
lgd = plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.xlabel("Day of forecast (k)")
plt.xticks(t_k_arr+1)
plt.ylabel("Mean absolute error (2007-2016)")

In [ ]:
X = clust_df.copy()
t_k = 4

X_t = pr.PoissonRegressionModel(t_k, []).add_autoregressive_col(X, t_k+1)

prm = pr.PoissonRegressionModel(t_k=t_k, covariates=['temperature', 'humidity', 'wind', 'rain'])
results, years = ev.cross_validation_years(prm, X_t)
print metrics.mean_absolute_error(*np.concatenate(results, axis=1))
print prm.fit_result.summary()

## Using old code

In [ ]:
# First, get predictors
import old_code.prediction.evaluation as ev_o
import old_code.prediction.cluster_regression as cr
reload(ev_o)
import pandas as pd
import numpy as np
from old_code.prediction.evaluation import kill_nanners
mets_baseline = dict()
mets_mem = dict()
mets_temphum = dict()
mets_weather = dict()
met = "MeanAbsErr"
clust_df_orig = load_integrated_df(os.path.join(DATA_DIR, 'archived/cluster/clust_feat_df_5.pkl'))
cdf_with_stuff = cr.add_autoreg_and_n_det(clust_df_orig.copy(), autoreg_cols=10, t_k_max=0, zero_padding=False)
cdf_with_stuff = kill_nanners(cdf_with_stuff.copy(), ["temp", "humidity", "wind", "rain_del_2"])

In [ ]:
max_t_k = 4

for t_k in range(max_t_k,-1,-1):
    legit_series = pd.Series(index=cdf_with_stuff.index)
    for clust in cdf_with_stuff.cluster.unique():
        clust_df = cdf_with_stuff[cdf_with_stuff.cluster==clust]
        legit_day = np.min(clust_df.dayofyear) + t_k
        legit_series[clust_df[clust_df.dayofyear >= legit_day].index] = 1
    y_t_k_base = cdf_with_stuff[legit_series == 1]['n_det']
    y_hat_t_k_base = np.exp(cdf_with_stuff[legit_series == 1]["autoreg_%d" % (1 + t_k)]) - 1

    # Baseline
    mets_baseline[t_k] = (ev_o.evaluate_glm(y_t_k_base, y_hat_t_k_base, metric=met))
    
    # models
    mets_mem[t_k] = ev_o.cross_validation_evaluation(cdf_with_stuff, autoreg=1, t_k=t_k, weather_vars=[], zero_padding=False, return_arrs=False, max_t_k=t_k, legit_series=legit_series, metrics=[met])
    mets_temphum[t_k] = ev_o.cross_validation_evaluation(cdf_with_stuff, autoreg=1, t_k=t_k, weather_vars=["temp","humidity"], zero_padding=False, return_arrs=False, max_t_k=t_k, legit_series=legit_series, metrics=[met])
    mets_weather[t_k] = ev_o.cross_validation_evaluation(cdf_with_stuff, autoreg=1, t_k=t_k, weather_vars=["temp","humidity","wind","rain_del_2"], zero_padding=False, return_arrs=False, max_t_k=t_k, legit_series=legit_series, metrics=[met])
    
    print "done with t_k %d" % t_k
    

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

n_t_k = 5
t_k_arr = np.arange(n_t_k)
mets = ["MeanAbsErr"]
i = 0
for met in mets:
    base_arr = np.array(map(lambda x: mets_baseline[x], t_k_arr))
    mem_arr = np.array(map(lambda x: mets_mem[x][i], t_k_arr))
    temphum_arr = np.array(map(lambda x: mets_temphum[x][i], t_k_arr))
    weather_arr = np.array(map(lambda x: mets_weather[x][i], t_k_arr))
    
    plt.plot(t_k_arr+1, base_arr, "kv--", label="Baseline", linewidth=2)
    plt.plot(t_k_arr+1, mem_arr, "gs--", label="Autoregression", linewidth=2)
    plt.plot(t_k_arr+1, temphum_arr, "r^--", label="Temp/hum", linewidth=2)
    plt.plot(t_k_arr+1, weather_arr, "bo--", label="All weather", linewidth=2)
    
    matplotlib.rcParams.update({'font.size': 14})
    lgd = plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.xlabel("Day of forecast (k)")
    plt.xticks(t_k_arr+1)
    plt.ylabel("Mean absolute error (2007-2016)")
    #plt.savefig("pics/mae.png", bbox_extra_artists=(lgd,), bbox_inches='tight')
    plt.show()
    plt.close()

## Now try old code with new data

In [ ]:
int_df_mod = int_df.copy()

# Modify new data to work with old code
int_df_mod['dayofyear'] = np.array(map(lambda x: du.dayofyear_from_datetime(x), int_df_mod.date_local))
int_df_mod = int_df_mod.assign(year=map(lambda x: x.year, int_df_mod.date_local))
int_df_mod.rename(columns={'temperature': 'temp', 'cluster_id': 'cluster', 'num_det': 'n_det'}, inplace=True)
int_df_mod['alt_cluster'] = np.empty(int_df_mod.shape[0]).fill(np.nan)
int_df_mod['rain_del_2'] = int_df_mod['rain']

In [ ]:
mets_baseline = dict()
mets_mem = dict()
mets_temphum = dict()
mets_weather = dict()
met = "MeanAbsErr"

#clust_df = load_integrated_df(os.path.join(DATA_DIR, 'archived/cluster/clust_feat_df_5.pkl'))

cdf_with_stuff_new = cr.add_autoreg_and_n_det(int_df_mod.copy(), autoreg_cols=10, t_k_max=0, zero_padding=False)
cdf_with_stuff_new = kill_nanners(cdf_with_stuff_new.copy(), ["temp", "humidity", "wind", "rain_del_2"])

In [ ]:
max_t_k = 4

for t_k in range(max_t_k,-1,-1):
    legit_series = pd.Series(index=cdf_with_stuff_new.index)
    for clust in cdf_with_stuff_new.cluster.unique():
        clust_df = cdf_with_stuff_new[cdf_with_stuff_new.cluster==clust]
        legit_day = np.min(int_df_mod.dayofyear) + (t_k+1)
        legit_series[clust_df[clust_df.dayofyear >= legit_day].index] = 1
    y_t_k_base = cdf_with_stuff_new[legit_series == 1]['n_det']
    y_hat_t_k_base = np.exp(cdf_with_stuff_new[legit_series == 1]["autoreg_%d" % (1 + t_k)]) - 1

    # Baseline
    mets_baseline[t_k] = (ev_o.evaluate_glm(y_t_k_base, y_hat_t_k_base, metric=met))
    
    # models
    mets_mem[t_k] = ev_o.cross_validation_evaluation(cdf_with_stuff_new[legit_series == 1], autoreg=1, t_k=t_k, weather_vars=[], zero_padding=False, return_arrs=False, max_t_k=t_k, legit_series=legit_series, metrics=[met])
    mets_temphum[t_k] = ev_o.cross_validation_evaluation(cdf_with_stuff_new[legit_series == 1], autoreg=1, t_k=t_k, weather_vars=["temp","humidity"], zero_padding=False, return_arrs=False, max_t_k=t_k, legit_series=legit_series, metrics=[met])
    mets_weather[t_k] = ev_o.cross_validation_evaluation(cdf_with_stuff_new[legit_series == 1], autoreg=1, t_k=t_k, weather_vars=["temp","humidity","wind","rain_del_2"], zero_padding=False, return_arrs=False, max_t_k=t_k, legit_series=legit_series, metrics=[met])
    
    print "done with t_k %d" % t_k
    

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

n_t_k = 5
t_k_arr = np.arange(n_t_k)
mets = ["MeanAbsErr"]
i = 0
for met in mets:
    base_arr = np.array(map(lambda x: mets_baseline[x], t_k_arr))
    mem_arr = np.array(map(lambda x: mets_mem[x][i], t_k_arr))
    temphum_arr = np.array(map(lambda x: mets_temphum[x][i], t_k_arr))
    weather_arr = np.array(map(lambda x: mets_weather[x][i], t_k_arr))
    
    plt.plot(t_k_arr+1, base_arr, "kv--", label="Baseline", linewidth=2)
    plt.plot(t_k_arr+1, mem_arr, "gs--", label="Autoregression", linewidth=2)
    plt.plot(t_k_arr+1, temphum_arr, "r^--", label="Temp/hum", linewidth=2)
    plt.plot(t_k_arr+1, weather_arr, "bo--", label="All weather", linewidth=2)
    
    matplotlib.rcParams.update({'font.size': 14})
    lgd = plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.xlabel("Day of forecast (k)")
    plt.xticks(t_k_arr+1)
    plt.ylabel("Mean absolute error (2007-2016)")
    #plt.savefig("pics/mae.png", bbox_extra_artists=(lgd,), bbox_inches='tight')
    plt.show()
    plt.close()